### Tools
Models can request to call tools that perform tasks such as fetching data from database, searching the web, or running code. 
Tools are pairing of:
1. A schema including the name of the tool, description, and parameters.
2. A function that implements the tool.

In [1]:
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:llama-3.1-8b-instant")

response=model.invoke("What is biocomputing in one line?")
response

AIMessage(content='Biocomputing is a field of research that uses biological molecules, such as DNA, proteins, and enzymes, as computational components to solve complex problems and process information.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 45, 'total_tokens': 80, 'completion_time': 0.056827144, 'completion_tokens_details': None, 'prompt_time': 0.003573937, 'prompt_tokens_details': None, 'queue_time': 0.069416133, 'total_time': 0.060401081}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd02a-9553-7251-bef8-a23c108660ec-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 45, 'output_tokens': 35, 'total_tokens': 80})

In [2]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather for a location."""
    return f"The weather in {location} is sunny."

model_with_tools=model.bind_tools([get_weather])

In [3]:
response=model_with_tools.invoke("Whats the weather in Bengaluru")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'teyt7t0wh', 'function': {'arguments': '{"location":"Bengaluru"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 218, 'total_tokens': 236, 'completion_time': 0.168299647, 'completion_tokens_details': None, 'prompt_time': 0.013755015, 'prompt_tokens_details': None, 'queue_time': 0.060889705, 'total_time': 0.182054662}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd02a-bd26-76e0-9313-f2485e2ef227-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Bengaluru'}, 'id': 'teyt7t0wh', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 218, 'output_tokens': 18, 'total_tokens': 236})

In [4]:
response.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Bengaluru'},
  'id': 'teyt7t0wh',
  'type': 'tool_call'}]

## Tools execution Loop

In [13]:
#Step 1: Model generates tool calls
messages=[{"role":"user","content":"Whats the weather in Bengaluru"}]
ai_msg=model_with_tools.invoke(messages)
messages.append(ai_msg)

#Step 2: Tool calls are executed and results are collected
for tool_call in ai_msg.tool_calls:
    tool_results=get_weather.invoke(tool_call)
    messages.append(tool_results)

#Step 3: Model processes the results and generates final response
final_response=model_with_tools.invoke(messages)
print(final_response.text)

In order to get the exact weather, I need to use the get_weather function.


In [14]:
messages

[{'role': 'user', 'content': 'Whats the weather in Bengaluru'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0vr4cr4tb', 'function': {'arguments': '{"location":"Bengaluru"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 218, 'total_tokens': 235, 'completion_time': 0.031977267, 'completion_tokens_details': None, 'prompt_time': 0.017081863, 'prompt_tokens_details': None, 'queue_time': 0.062796347, 'total_time': 0.04905913}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd02d-9c23-77b3-9da7-42c88d36bebf-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Bengaluru'}, 'id': '0vr4cr4tb', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 218, 'output_tokens': 17, 'total_tokens': 235}),
 ToolMessage(content='T